USLMCode(Australian Beard Tax (Promotion of Enlightenment Values) Act 1934)

{'inputs': [{'type': 'fact',
   'content': '{the suspected beard} was facial hair'},
  {'type': 'fact',
   'content': 'the length of the suspected beard was >= 5 millimetres'},
  {'type': 'fact',
   'content': 'the suspected beard occurred on or below the chin'}],
 'outputs': [{'type': 'fact',
   'content': 'the suspected beard was a beard',
   'name': 'the fact that the facial hair was a beard'}],
 'enactments': [{'source': '/au/act/1934/47/1/4/chapeau'},
  {'source': '/au/act/1934/47/1/4/a', 'suffix': ', or'}],
 'universal': True}

the Rule that the court MAY ALWAYS impose the
  RESULT:
    the Fact that <the suspected beard> was a beard
  GIVEN:
    the Fact that <the suspected beard> was facial hair
    the Fact that the length of <the suspected beard> was at least 5
    millimeter
    the Fact that <the suspected beard> occurred on or below the chin
  GIVEN the ENACTMENTS:
    "In this Act, beard means any facial hair no shorter than 5
    millimetres in length that:" (Australian Beard Tax (Promotion of
    Enlightenment Values) Act 1934, /au/act/1934/47/1/4/chapeau)
    "occurs on or below the chin" (Australian Beard Tax (Promotion of
    Enlightenment Values) Act 1934, /au/act/1934/47/1/4/a)

the Rule that the court MAY ALWAYS impose the
  RESULT:
    the Fact that <the suspected beard> was a beard
  GIVEN:
    the Fact that <the suspected beard> was facial hair
    the Fact that the length of <the suspected beard> was at least 5
    millimeter
    the Fact that <the suspected beard> existed in an uninterrup

the Rule that the court MAY ALWAYS impose the
  RESULT:
    the Fact that <the suspected beard> was a beard
  GIVEN:
    the Fact that <the suspected beard> was facial hair
    the Fact that the length of <the suspected beard> was exactly equal to
    8 millimeter
    the Fact that <the suspected beard> occurred on or below the chin
  GIVEN the ENACTMENTS:
    "In this Act, beard means any facial hair no shorter than 5
    millimetres in length that:" (Australian Beard Tax (Promotion of
    Enlightenment Values) Act 1934, /au/act/1934/47/1/4/chapeau)
    "occurs on or below the chin" (Australian Beard Tax (Promotion of
    Enlightenment Values) Act 1934, /au/act/1934/47/1/4/a)


True

the Rule that the court MUST ALWAYS impose the
  RESULT:
    the Fact it is false that <the suspected beard> was a beard
  GIVEN:
    the Fact that the length of <the suspected beard> was at least 12 inch
  DESPITE:
    the Fact that <the suspected beard> was facial hair
  GIVEN the ENACTMENTS:
    "In this Act, beard means any facial hair no shorter than 5
    millimetres in length that:" (Australian Beard Tax (Promotion of
    Enlightenment Values) Act 1934, /au/act/1934/47/1/4/chapeau)
    "exists in an uninterrupted line from the front of one ear to the
    front of the other ear below the nose." (Australian Beard Tax
    (Promotion of Enlightenment Values) Act 1934, /au/act/1934/47/1/4/b)


True

the Rule that the court MUST ALWAYS impose the
  RESULT:
    the Fact that <the defendant> committed the offense of improper
    transfer of beardcoin
  GIVEN:
    the Fact that <the beardcoin transaction> was a transfer of beardcoin
    between <the defendant> and <the counterparty>
    absence of the Fact that <the beardcoin transaction> was a licensed
    beardcoin repurchase
    the Fact it is false that <the counterparty> was <the Department of
    Beards>
  DESPITE:
    the Fact that the token attributed to <the Department of Beards>,
    asserting the fact that <the Department of Beards> granted an
    exemption from the prohibition of wearing beards, was counterfeit
  GIVEN the ENACTMENTS:
    "It shall be an offence to buy, sell, lend, lease, gift, transfer or
    receive in any way a beardcoin from any person or body other than the
    Department of Beards, except as provided in Part 4." (Australian Beard
    Tax (Promotion of Enlightenment Values) Act 1934,
    /au/act/1934/

the Rule that the court MUST ALWAYS impose the
  RESULT:
    the Fact that <the beardcoin transaction> was a transfer of beardcoin
    between <the defendant> and <the counterparty>
  GIVEN:
    the Fact that <the beardcoin transaction> was <the defendant>'s loan
    of the token attributed to <the Department of Beards>, asserting the
    fact that <the Department of Beards> granted an exemption from the
    prohibition of wearing beards, to <the counterparty>
  GIVEN the ENACTMENT:
    "It shall be an offence to buy, sell, lend, lease, gift, transfer or
    receive in any way a beardcoin from any person or body other than the
    Department of Beards, except as provided in Part 4." (Australian Beard
    Tax (Promotion of Enlightenment Values) Act 1934,
    /au/act/1934/47/3/7A)


the Rule that the court MUST ALWAYS impose the
  RESULT:
    the Fact that <the beardcoin transaction> was a transfer of beardcoin
    between <the defendant> and <the counterparty>
  GIVEN:
    the Fact that <the beardcoin transaction> was <the defendant>'s loan
    of the token attributed to <the Department of Beards>, asserting the
    fact that <the Department of Beards> granted an exemption from the
    prohibition of wearing beards, to <the counterparty>
    absence of the Fact that <the beardcoin transaction> was a licensed
    beardcoin repurchase
    the Fact it is false that <the counterparty> was <the Department of
    Beards>
  GIVEN the ENACTMENTS:
    "It shall be no defense to a charge under section 7A that the
    purchase, sale, lease, gift, transfer or receipt was of counterfeit
    beardcoin rather than genuine beardcoin." (Australian Beard Tax
    (Promotion of Enlightenment Values) Act 1934, /au/act/1934/47/3/7B/2)
    "It shall be an offence to buy, sell, lend, le

the Rule that the court MUST ALWAYS impose the
  RESULT:
    the Fact that <the beardcoin transaction> was a transfer of beardcoin
    between <the defendant> and <the counterparty>
    the Fact that <the defendant> committed the offense of improper
    transfer of beardcoin
  GIVEN:
    the Fact that <the beardcoin transaction> was <the defendant>'s loan
    of the token attributed to <the Department of Beards>, asserting the
    fact that <the Department of Beards> granted an exemption from the
    prohibition of wearing beards, to <the counterparty>
    absence of the Fact that <the beardcoin transaction> was a licensed
    beardcoin repurchase
    the Fact it is false that <the counterparty> was <the Department of
    Beards>
  GIVEN the ENACTMENTS:
    "It shall be no defense to a charge under section 7A that the
    purchase, sale, lease, gift, transfer or receipt was of counterfeit
    beardcoin rather than genuine beardcoin." (Australian Beard Tax
    (Promotion of Enlightenmen

14